In [1]:
import tensorflow as tf
import keras.backend as K
from keras.layers import LSTM, GRU, Dense, SimpleRNN
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [2]:
def split_sequence(sequence, n_steps):
  X = []
  Y = []
  for i in range(len(sequence)-n_steps):
    X.append(sequence[i:i+n_steps])
    Y.append(sequence[i+n_steps])
  return X, Y


In [3]:
datos = pd.read_excel('/content/drive/MyDrive/MIND/Iberdrola.xls')
scaler=StandardScaler()
datosN=scaler.fit_transform(datos['Último'].to_numpy().reshape(-1,1))
X, y = split_sequence(sequence=datosN, n_steps=20)
X=np.array(X)
y=np.array(y)

In [4]:
def porcentaje_margen(y_true, y_pred):
    margen = 0.02
    yy = K.sum(K.cast(K.less(K.abs((y_pred/y_true)-1.0),margen),dtype=float))
    return yy/K.cast(K.shape(y_pred)[0],dtype=float)

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=6)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

n_features=1
T=20

**LSTM**

In [24]:
model = Sequential()
model.add(LSTM(5, return_sequences=False, input_shape=(T, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[porcentaje_margen])
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=2)

Epoch 1/10
482/482 - 3s - loss: 0.1636 - porcentaje_margen: 0.0394 - 3s/epoch - 7ms/step
Epoch 2/10
482/482 - 2s - loss: 0.0257 - porcentaje_margen: 0.0726 - 2s/epoch - 4ms/step
Epoch 3/10
482/482 - 2s - loss: 0.0145 - porcentaje_margen: 0.0934 - 2s/epoch - 4ms/step
Epoch 4/10
482/482 - 2s - loss: 0.0125 - porcentaje_margen: 0.0705 - 2s/epoch - 4ms/step
Epoch 5/10
482/482 - 2s - loss: 0.0108 - porcentaje_margen: 0.1307 - 2s/epoch - 4ms/step
Epoch 6/10
482/482 - 2s - loss: 0.0105 - porcentaje_margen: 0.1328 - 2s/epoch - 4ms/step
Epoch 7/10
482/482 - 2s - loss: 0.0096 - porcentaje_margen: 0.1120 - 2s/epoch - 4ms/step
Epoch 8/10
482/482 - 2s - loss: 0.0091 - porcentaje_margen: 0.1266 - 2s/epoch - 4ms/step
Epoch 9/10
482/482 - 2s - loss: 0.0085 - porcentaje_margen: 0.1266 - 2s/epoch - 4ms/step
Epoch 10/10
482/482 - 2s - loss: 0.0080 - porcentaje_margen: 0.1266 - 2s/epoch - 4ms/step


In [25]:
pred_train = model.predict(X_train)
pred_train = scaler.inverse_transform(pred_train)
pred_test = model.predict(X_test)
pred_test = scaler.inverse_transform(pred_test)
testScore = porcentaje_margen(scaler.inverse_transform(y_test), pred_test)
print(f'Porcentaje de acierto en test: {testScore}')

Porcentaje de acierto en test: 0.8991596698760986


**GRU**

In [26]:
model = Sequential()
model.add(GRU(5, return_sequences=False, input_shape=(T, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[porcentaje_margen])
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=2)

Epoch 1/10
482/482 - 3s - loss: 0.2899 - porcentaje_margen: 0.0498 - 3s/epoch - 7ms/step
Epoch 2/10
482/482 - 2s - loss: 0.0219 - porcentaje_margen: 0.0913 - 2s/epoch - 4ms/step
Epoch 3/10
482/482 - 2s - loss: 0.0147 - porcentaje_margen: 0.0851 - 2s/epoch - 4ms/step
Epoch 4/10
482/482 - 2s - loss: 0.0110 - porcentaje_margen: 0.1141 - 2s/epoch - 4ms/step
Epoch 5/10
482/482 - 2s - loss: 0.0093 - porcentaje_margen: 0.1203 - 2s/epoch - 4ms/step
Epoch 6/10
482/482 - 2s - loss: 0.0083 - porcentaje_margen: 0.1349 - 2s/epoch - 4ms/step
Epoch 7/10
482/482 - 2s - loss: 0.0074 - porcentaje_margen: 0.1722 - 2s/epoch - 4ms/step
Epoch 8/10
482/482 - 2s - loss: 0.0071 - porcentaje_margen: 0.1473 - 2s/epoch - 4ms/step
Epoch 9/10
482/482 - 2s - loss: 0.0066 - porcentaje_margen: 0.1660 - 2s/epoch - 4ms/step
Epoch 10/10
482/482 - 2s - loss: 0.0064 - porcentaje_margen: 0.1577 - 2s/epoch - 4ms/step


In [27]:
pred_train = model.predict(X_train)
pred_train = scaler.inverse_transform(pred_train)
pred_test = model.predict(X_test)
pred_test = scaler.inverse_transform(pred_test)
testScore = porcentaje_margen(scaler.inverse_transform(y_test), pred_test)
print(f'Porcentaje de acierto en test: {testScore}')

Porcentaje de acierto en test: 0.9075630307197571


**SimpleRNN**

In [28]:
model = Sequential()
model.add(SimpleRNN(5, return_sequences=False, input_shape=(T, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[porcentaje_margen])
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=2)

Epoch 1/10
482/482 - 2s - loss: 0.6851 - porcentaje_margen: 0.0124 - 2s/epoch - 4ms/step
Epoch 2/10
482/482 - 1s - loss: 0.1553 - porcentaje_margen: 0.0353 - 1s/epoch - 3ms/step
Epoch 3/10
482/482 - 1s - loss: 0.0536 - porcentaje_margen: 0.0622 - 1s/epoch - 3ms/step
Epoch 4/10
482/482 - 1s - loss: 0.0303 - porcentaje_margen: 0.0643 - 1s/epoch - 3ms/step
Epoch 5/10
482/482 - 1s - loss: 0.0244 - porcentaje_margen: 0.0560 - 1s/epoch - 3ms/step
Epoch 6/10
482/482 - 1s - loss: 0.0213 - porcentaje_margen: 0.0871 - 1s/epoch - 3ms/step
Epoch 7/10
482/482 - 1s - loss: 0.0175 - porcentaje_margen: 0.0851 - 1s/epoch - 3ms/step
Epoch 8/10
482/482 - 1s - loss: 0.0164 - porcentaje_margen: 0.1079 - 1s/epoch - 3ms/step
Epoch 9/10
482/482 - 1s - loss: 0.0149 - porcentaje_margen: 0.0996 - 1s/epoch - 3ms/step
Epoch 10/10
482/482 - 1s - loss: 0.0129 - porcentaje_margen: 0.0809 - 1s/epoch - 3ms/step


In [29]:
pred_train = model.predict(X_train)
pred_train = scaler.inverse_transform(pred_train)
pred_test = model.predict(X_test)
pred_test = scaler.inverse_transform(pred_test)
testScore = porcentaje_margen(scaler.inverse_transform(y_test), pred_test)
print(f'Porcentaje de acierto en test: {testScore}')

Porcentaje de acierto en test: 0.8109243512153625
